In [1]:
import pickle

import pandas as pd
import numpy as np
import networkx as nx
from scipy.stats import beta
import itertools

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import rankdata, kendalltau, spearmanr

from tqdm.notebook import tqdm
from pathlib import Path

from itertools import zip_longest

sns.set_theme("paper", "white", rc={'text.usetex' : True})

In [2]:
df_users = pd.read_csv("./DSock/Active_Users.csv", index_col=0)
display(df_users.head())

timeline = pd.date_range("2020-09-03-03:59:59", periods=6, tz="utc")

,MTurk_ID,username,id,isMod,isObserver,isPuppet,n_reports,puppet,report,issued_posts,...,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Strategy
1,A10IUXCZBNYUXC,Jacob_1907,2658,f,f,f,17,0,1,25,...,2.0,2.0,2.0,1.0,1.0,2.0,1.0,4.0,2.0,NaN
2,A15F2JTVAJXITI,Luke_3376,2297,f,t,f,0,0,0,0,...,1.0,3.0,1.0,3.0,1.0,3.0,1.0,4.0,1.0,NaN
3,A173A97OFDAX9F,Claire_679,2601,f,f,f,0,0,0,4,...,1.0,2.0,2.0,1.0,1.0,2.0,1.0,3.0,3.0,NaN
4,A18XVBWB2A270I,Katherine_2323,2338,f,t,f,0,0,0,0,...,1.0,2.0,4.0,2.0,2.0,2.0,1.0,1.0,3.0,NaN
5,A1AIMKA54UBYIQ,Asia_4906,2638,f,f,f,1,0,1,9,...,3.0,3.0,2.0,1.0,2.0,2.0,2.0,4.0,3.0,NaN


In [3]:
topics = list("abcdefgh")
nums = list(range(9))
num2topic = dict(zip(nums, topics))
topic2num = dict(zip(topics, nums))
print(f"{topic2num} {num2topic}")
deltas = ["10", "21", "32", "43", "54"]

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7} {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h'}


In [4]:
df_evals = pd.read_csv("./DSock/human_eval_analysis/attitude_tracking.csv")
df_evals = pd.merge(df_users, df_evals, how="left", left_on="MTurk_ID", right_on="id").rename({"id_x": "id"}, axis=1)
df_evals.index = [f"u{u}" for u in df_evals["id"]]

display(df_evals.head())

get_change = lambda u, t: df_evals.loc[u][[f"{t}_delta_{d}" for d in deltas]].tolist()
print(get_change("u2337", num2topic[0]))

,MTurk_ID,username,id,isMod,isObserver,isPuppet,n_reports,puppet,report,issued_posts,...,g_delta_10,g_delta_21,g_delta_32,g_delta_43,g_delta_54,h_delta_10,h_delta_21,h_delta_32,h_delta_43,h_delta_54
u2658,A10IUXCZBNYUXC,Jacob_1907,2658,f,f,f,17,0,1,25,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
u2297,A15F2JTVAJXITI,Luke_3376,2297,f,t,f,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
u2601,A173A97OFDAX9F,Claire_679,2601,f,f,f,0,0,0,4,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
u2338,A18XVBWB2A270I,Katherine_2323,2338,f,t,f,0,0,0,0,...,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
u2638,A1AIMKA54UBYIQ,Asia_4906,2638,f,f,f,1,0,1,9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


[3.0, 0.0, 0.0, 0.0, 0.0]


In [5]:
df_posts = pd.read_csv("./DSock/direct_influence/posts_with_senti_and_topic.csv", parse_dates=["createdAt"])
df_comts = pd.read_csv("./DSock/direct_influence/comments_with_senti_and_topic.csv", delimiter=",", parse_dates=["createdAt"]).dropna(subset=["CommenterId", "PostId"])
df_views = pd.read_csv("./DSock/postViews.csv", delimiter="|", parse_dates=["createdAt"]).dropna(subset=["UserId", "PostId"])
df_likes = pd.read_csv("./DSock/Likes.csv", delimiter="|", parse_dates=["createdAt"]).dropna(subset=["UserId", "PostId"])
df_repts = pd.read_csv("./DSock/ReportUsers.csv", delimiter="|", parse_dates=["createdAt"])

print(f"posts: {df_posts.shape}, comments: {df_comts.shape}, views: {df_views.shape}, likes: {df_likes.shape}, reports: {df_repts.shape}")

df_likes["UserId"] = df_likes["UserId"].astype(int)
df_likes["PostId"] = df_likes["PostId"].astype(int)

df_comts["CommenterId"] = df_comts["CommenterId"].astype(int)
df_comts["PostId"] = df_comts["PostId"].astype(int)

df_annot_posts = pd.read_csv("./DSock/posts_anno_labels.csv")
df_posts = df_posts.merge(df_annot_posts[["post_id", "majority_topic_label", "majority_sent_label"]], left_on="PostID", right_on="post_id")

df_annot_comts = pd.read_csv("./DSock/comments_anno_labels.csv")
df_comts = df_comts.merge(df_annot_comts[["comment_id", "majority_topic_label", "majority_sent_label"]], left_on="id", right_on="comment_id")

posts: (1057, 11), comments: (1615, 11), views: (234178, 10), likes: (1034, 6), reports: (46, 6)


In [6]:
post_comments = {f"p{p}": [] for p in df_posts["PostID"].values}
for c, p in df_comts[["id", "PostId"]].values:
    if f"p{p}" in post_comments:
        post_comments[f"p{p}"].append(f"c{c}")
G = nx.DiGraph()
for u, uname, obsr, sock in df_users[["id", "username", "isObserver", "isPuppet"]].values:
    G.add_node(f"u{u}", id=u, kind="user", name=uname, observer=(obsr == "t"), sock=(sock == "t"))
    if obsr == "t":
        G.nodes[f"u{u}"]["color"] = "obsr"
    elif sock == "t":
        G.nodes[f"u{u}"]["color"] = "sock"
    else:
        G.nodes[f"u{u}"]["color"] = "part"

for p, u, t, polar, score, topic in df_posts[["PostID", "AuthorId", "createdAt", "majority_sent_label", "scores", "majority_topic_label"]].values:
    if f"u{u}" in G.nodes:
        G.add_node(f"p{p}", id=p, kind="post", time=t, user=u, polar=polar, score=score, topic=topic)

for c, u, p, t, polar, score, topic in df_comts[["id", "CommenterId", "PostId", "createdAt", "majority_sent_label", "scores", "majority_topic_label"]].values:
    if f"u{u}" in G.nodes and f"p{p}" in G.nodes:
        G.add_node(f"c{c}", id=c, kind="comt", time=t, user=u, polar=polar, score=score, topic=topic)
        G.add_edge(f"c{c}", f"p{p}", kind="known", time=t, weight=1)
        # G.add_edge(f"p{p}", f"c{c}", kind="known", time=t, weight=1)

for l, u, p, t in df_likes[["id", "UserId", "PostId", "createdAt"]].values:
    if f"u{u}" in G.nodes and f"p{p}" in G.nodes:
        G.add_node(f"l{l}", user=f"u{u}", post=f"p{p}", time=t, id=l, kind="like",
                   topic=G.nodes[f"p{p}"]["topic"], polar=G.nodes[f"p{p}"]["polar"], score=1)
        G.add_edge(f"u{u}", f"l{l}", time=t, weight=1, kind="infer")
        G.add_edge(f"l{l}", f"p{p}", time=t, weight=1, kind="known")

for u, p, t, v, s in df_views[["UserId", "PostId", "createdAt", "id", "singleView"]].values:
    if f"u{u}" in G.nodes and f"p{p}" in G.nodes:
        G.add_node(f"v{v}", user=f"u{u}", post=f"p{p}", time=t, id=v, kind="view",
                   topic=G.nodes[f"p{p}"]["topic"], polar=G.nodes[f"p{p}"]["polar"], score=1)
        G.add_edge(f"u{u}", f"v{v}", time=t, weight=1, kind="infer")
        G.add_edge(f"v{v}", f"p{p}", time=t, weight=1, kind="infer")
        if s == "t":
            for c in post_comments[f"p{p}"]:
                if c in G.nodes:
                    G.add_edge(f"v{v}", c, time=t, weight=1, kind="infer")
user_view = {u: set() for u in G if u[0] == "u"}
for u, p, t, v, s in df_views[["UserId", "PostId", "createdAt", "id", "singleView"]].values:
    if f"u{u}" in G.nodes and f"p{p}" in G.nodes and s != "t":
        user_view[f"u{u}"].add(f"v{v}")

single_view = {u: set() for u in G if u[0] == "u"}
for u, p, t, v, s in df_views[["UserId", "PostId", "createdAt", "id", "singleView"]].values:
    if f"u{u}" in G.nodes and f"p{p}" in G.nodes and s == "t":
        single_view[f"u{u}"].add(f"v{v}")

print(f"user view: {sum([len(user_view[u]) for u in user_view])}")
print(f"single view: {sum([len(single_view[u]) for u in single_view])}")

for p in tqdm(G):
    if p[0] in ["p", "c"]:
        u = f"u{G.nodes[p]['user']}"
        for v in user_view[u] | single_view[u]:
            if G.nodes[v]["time"] <= G.nodes[p]["time"]:
                G.add_edge(p, v, time=G.nodes[v]["time"], weight=1, kind="infer")

user view: 210543
single view: 3808


  0%|          | 0/218011 [00:00<?, ?it/s]

In [7]:
with open("res/pagerank/0.85-0.5-0.5.pkl", "rb") as fp:
    d = pickle.load(fp)

with open("res/pagerank/fig2.pkl", "rb") as fp:
    fig2_value = pickle.load(fp)

In [8]:
df_users[df_users["id"] == 2340]

,MTurk_ID,username,id,isMod,isObserver,isPuppet,n_reports,puppet,report,issued_posts,...,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Strategy
7,A1CGW3DQ5JRSJH,Asia_3576,2340,f,t,f,0,0,0,0,...,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,3.0,NaN


In [9]:
user_list = df_users["id"].tolist()
# user_list = df_users[df_users["isObserver"] == "f"]["id"].tolist()

In [10]:
obsr_list = [f"u{u}" for u in df_users[df_users["isObserver"] == "t"]["id"]]
pr_alpha = 0.85
beta_a = 0.5
beta_b = 0.5
keyq = [(node_name, t, topic, polar, pr_alpha, beta_a, beta_b)
        for node_name in obsr_list for t in range(1,6) for topic in range(8) for polar in [0, 2]
       ]

keyq = list(filter(lambda x: get_change(x[0], num2topic[x[2]])[x[1]-1]*(1-x[3]) > 0, tqdm(keyq)))

  0%|          | 0/2720 [00:00<?, ?it/s]

In [11]:
property_full = {k: d[k]["full_graph"] for k in tqdm(keyq)}
property_exps = {k: d[k]["exps_graph"] for k in tqdm(keyq)}

df_full = pd.DataFrame.from_dict(property_full, orient="index")
df_exps = pd.DataFrame.from_dict(property_exps, orient="index")

  0%|          | 0/142 [00:00<?, ?it/s]

  0%|          | 0/142 [00:00<?, ?it/s]

In [12]:
use_color = sns.color_palette("Set1")
use_color

[(0.8941176470588236, 0.10196078431372549, 0.10980392156862745),
 (0.21568627450980393, 0.49411764705882355, 0.7215686274509804),
 (0.30196078431372547, 0.6862745098039216, 0.2901960784313726),
 (0.596078431372549, 0.3058823529411765, 0.6392156862745098),
 (1.0, 0.4980392156862745, 0.0),
 (1.0, 1.0, 0.2),
 (0.6509803921568628, 0.33725490196078434, 0.1568627450980392),
 (0.9686274509803922, 0.5058823529411764, 0.7490196078431373),
 (0.6, 0.6, 0.6)]

In [13]:
pd.DataFrame.from_dict({
    ("Full Influence Graph", "mean"): df_full.mean(axis=0),
    ("Full Influence Graph", "std"): df_full.std(axis=0),
    ("Expression only Influence Graph", "mean"): df_exps.mean(axis=0),
    ("Expression only Influence Graph", "std"): df_exps.std(axis=0),
}).T

nodes        edges  components  \
Full Influence Graph            mean   462.183099  1476.161972    1.000000   
                                std   1225.934304  4455.952356    0.000000   
Expression only Influence Graph mean    14.711268     5.619718    9.091549   
                                std     26.742158    12.095209   15.051168   

                                       density  diameter    degree  
Full Influence Graph            mean  0.023673  3.021127  1.305359  
                                std   0.059780  2.980952  1.273526  
Expression only Influence Graph mean  0.008994  0.000000  0.149261  
                                std   0.018341  0.000000  0.208950

In [14]:
pab = [(p, a, b)
    for p in [0.85, 0.9, 0.7, 0.5, 0.3, 0.1]
    for a in [0.5, 1.0, 1.5, 2.0]
    for b in [0.5, 1.0, 1.5, 2.0]
    # for a in [0.5, 0.9, 0.7, 0.3, 0.1]
    # for b in [0.5, 0.9, 0.7, 0.3, 0.1]
]
obsr_list = [f"u{u}" for u in df_users[df_users["isObserver"] == "t"]["id"]]

In [15]:
df_obsr_express = pd.read_csv("./DSock/Observer_Expressed_Influence.csv")
df_obsr_express.fillna(-1, inplace=True)
df_obsr_express["influence_1"] = df_obsr_express["influence_1"].astype(int)
df_obsr_express["influence_2"] = df_obsr_express["influence_2"].astype(int)
df_obsr_express["influence_3"] = df_obsr_express["influence_3"].astype(int)
df_obsr_express["influence_4"] = df_obsr_express["influence_4"].astype(int)
print(df_obsr_express.shape)
display(df_obsr_express.head())

(39, 8)


,MTurk_ID,id,Role,survey,influence_1,influence_2,influence_3,influence_4
0,A3L5XFNSOY6QET,2360,observer,1,2704,2407,-1,-1
1,A3FJ3XMXO8UAJ0,2339,observer,1,2704,2540,2593,-1
2,A35AMS7DP33RQZ,2308,observer,1,2454,-1,-1,-1
3,AEOVYZ83Q3GX4,2301,observer,1,2658,2543,-1,-1
4,A2HJYX70UQW9XR,2330,observer,1,2658,2704,2667,-1


In [16]:
obsr_dict = {
    node_name: {
        t: [(topic, polar) for topic in range(8) for polar in [0, 2]
#             if get_change(node_name, num2topic[topic])[t - 1]*(1-polar) > 0
           ]
        for t in range(1, 6)
    }
    for node_name in obsr_list
}

In [17]:
def fig1(p, a, b):
    fpath = Path(f"res/pagerank/{p}-{a}-{b}.pkl")
    if not fpath.exists():
        return None
    with open(fpath, "rb") as fp:
        d = pickle.load(fp)
    rank_dict = []
    for u, s, inf1, inf2, inf3, inf4 in df_obsr_express[["id", "survey", "influence_1", "influence_2" , "influence_3", "influence_4"]].values:
        infs = list(filter(lambda x: x > 0, [inf1, inf2, inf3, inf4]))
        uname = f"u{u}"
        ret_dict = {"id": u, "survey": s}
        if uname in obsr_dict:
            # p, a, b = 0.85, 0.5, 0.5
            inf_u = [u for u in infs if u in user_list]
            inf_dict = {u: np.nan for u in inf_u}
            which_dict = {u: np.nan for u in inf_u}
            for topic, polar in obsr_dict[uname][s]:
                pr_value = d[(uname, s, topic, polar, p, a, b)]["pr"]
                ret = {u: 0 for u in user_list}
                for n in pr_value:
                    if n[0] in ["p", "c"]:
                        ret[G.nodes[n]["user"]] += pr_value[n]
                rank_value = [-ret[u] for u in user_list]
                # rank_value = [v if v > 0 else np.nan for v in rank_value]
                rank = rankdata(rank_value, "max")
                # rank = [r if r < max(rank) else np.nan for r in rank]
                rdict = dict(zip(user_list, rank))
                inf_v = [rdict[u] for u in inf_u]
                inf_r = dict(zip(inf_u, inf_v))
                for u in inf_dict:
                    which_dict[u] = (topic, polar) if inf_r[u] < inf_dict[u] or np.isnan(inf_dict[u]) else which_dict[u]
                    inf_dict[u] = inf_r[u] if inf_r[u] < inf_dict[u] or np.isnan(inf_dict[u]) else inf_dict[u]

            rank_dict += [
                {
                    **ret_dict,
                    **dict(zip_longest(range(1, 5), [inf_dict[u] for u in inf_u])),
                    **dict(zip_longest([f"which_{i}" for i in range(1, 5)], [which_dict[u] for u in inf_u])),
                }
            ]
    return rank_dict

def fig2():
    fpath = Path(f"res/pagerank/fig2.pkl")
    if not fpath.exists():
        return None
    with open(fpath, "rb") as fp:
        d = pickle.load(fp)
    rank_dict = []
    for u, s, inf1, inf2, inf3, inf4 in df_obsr_express[["id", "survey", "influence_1", "influence_2" , "influence_3", "influence_4"]].values:
        infs = list(filter(lambda x: x > 0, [inf1, inf2, inf3, inf4]))
        uname = f"u{u}"
        ret_dict = {"id": u, "survey": s}
        if uname in obsr_dict:
            inf_u = [u for u in infs if u in user_list]
            inf_dict = {u: np.nan for u in inf_u}
            which_dict = {u: np.nan for u in inf_u}
            for topic, polar in obsr_dict[uname][s]:
                pr_value = d[(uname, s, topic, polar)]["fig2"]
                ret = {u: 0 for u in user_list}
                for n in pr_value:
                    if n[0] in ["p", "c"]:
                        ret[G.nodes[n]["user"]] += pr_value[n]
                rank_value = [-ret[u] for u in user_list]
                # rank_value = [v if v > 0 else np.nan for v in rank_value]
                rank = rankdata(rank_value, "max")
                # rank = [r if r < max(rank) else np.nan for r in rank]
                rdict = dict(zip(user_list, rank))
                inf_v = [rdict[u] for u in inf_u]
                inf_r = dict(zip(inf_u, inf_v))
                for u in inf_dict:
                    which_dict[u] = (topic, polar) if inf_r[u] < inf_dict[u] or np.isnan(inf_dict[u]) else which_dict[u]
                    inf_dict[u] = inf_r[u] if inf_r[u] < inf_dict[u] or np.isnan(inf_dict[u]) else inf_dict[u]

            rank_dict += [
                {
                    **ret_dict,
                    **dict(zip_longest(range(1, 5), [inf_dict[u] for u in inf_u])),
                    **dict(zip_longest([f"which_{i}" for i in range(1, 5)], [which_dict[u] for u in inf_u])),
                }
            ]
    return rank_dict

def fig3(p, a, b):
    with open("res/pagerank/fig2.pkl", "rb") as fp:
        fig2_value = pickle.load(fp)
    fpath = Path(f"res/pagerank/{p}-{a}-{b}.pkl")
    if not fpath.exists():
        return None
    with open(fpath, "rb") as fp:
        d = pickle.load(fp)
    rank_dict = []
    for u, s, inf1, inf2, inf3, inf4 in df_obsr_express[["id", "survey", "influence_1", "influence_2" , "influence_3", "influence_4"]].values:
        infs = list(filter(lambda x: x > 0, [inf1, inf2, inf3, inf4]))
        uname = f"u{u}"
        ret_dict = {"id": u, "survey": s}
        if uname in obsr_dict:
            # p, a, b = 0.85, 0.5, 0.5
            inf_u = [u for u in infs if u in user_list]
            inf_dict = {u: np.nan for u in inf_u}
            which_dict = {u: np.nan for u in inf_u}
            for topic, polar in obsr_dict[uname][s]:
                pr_value = d[(uname, s, topic, polar, p, a, b)]["pr"]
                ret = {u: 0 for u in user_list}
                for n in pr_value:
                    if n[0] in ["p", "c"]:
                        ret[G.nodes[n]["user"]] += pr_value[n] * fig2_value[(uname, s, topic, polar)]["fig2"][n]
                rank_value = [-ret[u] for u in user_list]
                # rank_value = [v if v > 0 else np.nan for v in rank_value]
                rank = rankdata(rank_value, "max")
                # rank = [r if r < max(rank) else np.nan for r in rank]
                rdict = dict(zip(user_list, rank))
                inf_v = [rdict[u] for u in inf_u]
                inf_r = dict(zip(inf_u, inf_v))
                for u in inf_dict:
                    which_dict[u] = (topic, polar) if inf_r[u] < inf_dict[u] or np.isnan(inf_dict[u]) else which_dict[u]
                    inf_dict[u] = inf_r[u] if inf_r[u] < inf_dict[u] or np.isnan(inf_dict[u]) else inf_dict[u]

            rank_dict += [
                {
                    **ret_dict,
                    **dict(zip_longest(range(1, 5), [inf_dict[u] for u in inf_u])),
                    **dict(zip_longest([f"which_{i}" for i in range(1, 5)], [which_dict[u] for u in inf_u])),
                }
            ]
    return rank_dict

In [18]:
df_res_fig1 = pd.DataFrame.from_records(fig1(0.85, 0.5, 0.5))
display(df_res_fig1.head())

df_res_fig2 = pd.DataFrame.from_records(fig2())
display(df_res_fig2.head())

df_res_fig3 = pd.DataFrame.from_records(fig3(0.85, 0.5, 0.5))
display(df_res_fig3.head())

,id,survey,1,2,3,4,which_1,which_2,which_3,which_4
0,2360,1,116,NaN,NaN,NaN,"(0, 0)",None,None,None
1,2339,1,116,2.0,1.0,NaN,"(0, 0)","(1, 0)","(1, 2)",None
2,2308,1,116,NaN,NaN,NaN,"(0, 0)",None,None,None
3,2301,1,116,116.0,NaN,NaN,"(0, 0)","(0, 0)",None,None
4,2330,1,116,116.0,3.0,NaN,"(0, 0)","(0, 0)","(2, 2)",None


,id,survey,1,2,3,4,which_1,which_2,which_3,which_4
0,2360,1,116,NaN,NaN,NaN,"(0, 0)",None,None,None
1,2339,1,116,3.0,1.0,NaN,"(0, 0)","(1, 0)","(1, 2)",None
2,2308,1,116,NaN,NaN,NaN,"(0, 0)",None,None,None
3,2301,1,116,116.0,NaN,NaN,"(0, 0)","(0, 0)",None,None
4,2330,1,116,116.0,3.0,NaN,"(0, 0)","(0, 0)","(2, 2)",None


,id,survey,1,2,3,4,which_1,which_2,which_3,which_4
0,2360,1,116,NaN,NaN,NaN,"(0, 0)",None,None,None
1,2339,1,116,2.0,1.0,NaN,"(0, 0)","(1, 0)","(1, 2)",None
2,2308,1,116,NaN,NaN,NaN,"(0, 0)",None,None,None
3,2301,1,116,116.0,NaN,NaN,"(0, 0)","(0, 0)",None,None
4,2330,1,116,116.0,3.0,NaN,"(0, 0)","(0, 0)","(2, 2)",None


### TOP K precision

For the top K in FIGRank, how many match the most influential by human evaluation

For example, one obse nominated account x, y, z in a survey

For all the FIGs of the observer including all topics and polarity, topk(x) is the best ranking of x.

In [19]:
topk = lambda df, k: (df[range(1, 5)] <= k).sum().sum() / (~np.isnan(df[range(1, 5)])).sum().sum()

In [20]:
pab = [(p, a, b)
    for p in [0.85, 0.9, 0.7, 0.5, 0.3, 0.1]
    for a in [0.5, 1.0, 1.5, 2.0]
    for b in [0.5, 1.0, 1.5, 2.0]
    # for a in [0.5, 0.9, 0.7, 0.3, 0.1]
    # for b in [0.5, 0.9, 0.7, 0.3, 0.1]
]

res_fig2 = {"fig2": fig2()}
res_fig1 = {(p, a, b): fig1(p, a, b) for p, a, b in tqdm(pab)}
res_fig3 = {(p, a, b): fig3(p, a, b) for p, a, b in tqdm(pab)}

  0%|          | 0/96 [00:00<?, ?it/s]

  0%|          | 0/96 [00:00<?, ?it/s]

In [21]:
eval_fig1 = {
    (p, a, b): {
        k: topk(pd.DataFrame.from_records(res_fig1[(p,a,b)]), k)
        for k in [1, 2, 3, 4, 5, 10, 15, 20, 30, 40, 50, 100, 116]
    }
    for p, a, b in tqdm(res_fig1.keys())
}

eval_fig2 = {
    key: {
        k: topk(pd.DataFrame.from_records(res_fig2[key]), k)
        for k in [1, 2, 3, 4, 5, 10, 15, 20, 30, 40, 50, 100, 116]
    }
    for key in tqdm(res_fig2.keys())
}

eval_fig3 = {
    (p, a, b): {
        k: topk(pd.DataFrame.from_records(res_fig3[(p,a,b)]), k)
        for k in [1, 2, 3, 4, 5, 10, 15, 20, 30, 40, 50, 100, 116]
    }
    for p, a, b in tqdm(res_fig3.keys())
}

  0%|          | 0/96 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/96 [00:00<?, ?it/s]

In [22]:
report_fig1 = pd.DataFrame.from_dict(eval_fig1, orient="index")
idxmax = report_fig1.idxmax()
display(pd.concat([report_fig1.loc[ind, :] for ind in set(idxmax)], axis=1).T)

1         2         3         4         5         10   \
0.10 1.0 0.5  0.239130  0.413043  0.554348  0.619565  0.630435  0.771739   
0.85 1.5 0.5  0.195652  0.391304  0.521739  0.576087  0.608696  0.771739   
     0.5 0.5  0.195652  0.391304  0.500000  0.576087  0.608696  0.771739   
0.10 0.5 1.0  0.239130  0.413043  0.510870  0.597826  0.641304  0.771739   
         0.5  0.239130  0.413043  0.532609  0.619565  0.630435  0.771739   
0.90 0.5 1.5  0.250000  0.391304  0.478261  0.554348  0.619565  0.771739   

                   15        20        30        40        50        100  116  
0.10 1.0 0.5  0.782609  0.782609  0.793478  0.793478  0.793478  0.793478  1.0  
0.85 1.5 0.5  0.782609  0.793478  0.793478  0.793478  0.793478  0.793478  1.0  
     0.5 0.5  0.782609  0.782609  0.793478  0.793478  0.793478  0.793478  1.0  
0.10 0.5 1.0  0.782609  0.782609  0.793478  0.793478  0.793478  0.793478  1.0  
         0.5  0.782609  0.782609  0.793478  0.793478  0.793478  0.793478  1.0  
0.90 0.5 1.5  0.782609  0.782609  0.793478  0.793478  0.793478  0.793478  1.0

In [23]:
report_fig2 = pd.DataFrame.from_dict(eval_fig2, orient="index")
idxmax = report_fig2.idxmax()
display(pd.concat([report_fig2.loc[ind, :] for ind in set(idxmax)], axis=1).T)

,1,2,3,4,5,10,15,20,30,40,50,100,116
fig2,0.195652,0.347826,0.456522,0.543478,0.586957,0.706522,0.76087,0.771739,0.793478,0.793478,0.793478,0.793478,1.0


In [24]:
report_fig3 = pd.DataFrame.from_dict(eval_fig3, orient="index")
idxmax = report_fig3.idxmax()
display(pd.concat([report_fig3.loc[ind, :] for ind in set(idxmax)], axis=1).T)

1         2         3         4         5         10   \
0.85 1.5 0.5  0.239130  0.402174  0.510870  0.576087  0.608696  0.782609   
0.70 0.5 0.5  0.250000  0.380435  0.510870  0.565217  0.619565  0.782609   
0.85 0.5 0.5  0.228261  0.391304  0.510870  0.554348  0.619565  0.782609   
     1.0 0.5  0.228261  0.402174  0.510870  0.565217  0.608696  0.782609   
0.30 0.5 0.5  0.239130  0.391304  0.532609  0.565217  0.641304  0.771739   
0.50 2.0 0.5  0.239130  0.402174  0.510870  0.565217  0.641304  0.782609   

                   15        20        30        40        50        100  116  
0.85 1.5 0.5  0.793478  0.793478  0.793478  0.793478  0.793478  0.793478  1.0  
0.70 0.5 0.5  0.782609  0.782609  0.793478  0.793478  0.793478  0.793478  1.0  
0.85 0.5 0.5  0.782609  0.793478  0.793478  0.793478  0.793478  0.793478  1.0  
     1.0 0.5  0.782609  0.793478  0.793478  0.793478  0.793478  0.793478  1.0  
0.30 0.5 0.5  0.782609  0.782609  0.793478  0.793478  0.793478  0.793478  1.0  
0.50 2.0 0.5  0.782609  0.793478  0.793478  0.793478  0.793478  0.793478  1.0